In [1]:
!pip install seqeval

In [2]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [5]:
# verify GPU
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


## Convert to DataFrame for EDA

In [6]:
def convert_data(filepath):

  # Read the data from the text file
  with open(filepath, "r") as file:
      lines = file.readlines()

  # Define an empty list to store the data
  data = []

  # Iterate over each line in the file
  for line in lines:
      # Split the line by spaces
      parts = line.strip().split()

      # Check if the line has the expected number of elements
      if len(parts) == 9:
          # Extract the values from the line
          text_file_name = parts[0]
          sentence_line_number = int(parts[1])
          sentence_word_index = int(parts[2])
          sentence_seq = parts[3]
          start_token = int(parts[4])
          end_token = int(parts[5])
          original_word = parts[6]
          word = parts[7]
          label = parts[8]

          # Append the values as a tuple to the data list
          data.append((text_file_name, sentence_line_number, sentence_word_index, sentence_seq,
                      start_token, end_token, original_word, word, label))

  # Create a DataFrame from the data list with appropriate column names
  df = pd.DataFrame(data, columns=['text_file_name', 'sentence_line_number', 'sentence_word_index',
                                  'sentence_seq', 'start_token', 'end_token', 'original_word',
                                  'word', 'label'])

  return df


In [7]:
train_data_path = "/content/drive/MyDrive/266_final/data/Original_text/dataset1_train.txt"
test_data_path = "/content/drive/MyDrive/266_final/data/Original_text/dataset1_test.txt"

train = convert_data(train_data_path)
test = convert_data(test_data_path)

print(f"Length of train: {len(train)}")
print(f"Length of test: {len(test)}")


Length of train: 895141
Length of test: 585761


In [8]:
df = train.copy()

In [9]:
df['sentence_line_number'].nunique()

1053

In [10]:
df.head()

,text_file_name,sentence_line_number,sentence_word_index,sentence_seq,start_token,end_token,original_word,word,label
0,data/training_20180910/110727.txt,1,0,NA,0,9,Admission,Admission,O
1,data/training_20180910/110727.txt,1,1,NA,10,14,Date,Date,O
2,data/training_20180910/110727.txt,1,2,NA,14,15,:,:,O
3,data/training_20180910/110727.txt,1,3,NA,17,18,[,[,O
4,data/training_20180910/110727.txt,1,4,NA,18,19,*,*,O


In [11]:
df['label'].value_counts()

label
O              802045
B-Drug          16222
I-Frequency     13023
I-Dosage         8779
B-Strength       6691
B-Form           6647
I-Strength       6617
B-Frequency      6279
B-Route          5475
I-Drug           4298
B-Dosage         4221
I-Form           4173
B-Reason         3791
I-Reason         3125
I-Duration       1034
B-ADE             956
I-ADE             776
B-Duration        592
I-Route           397
Name: count, dtype: int64

### Label labels

In [13]:
labels = [i.split() for i in df['label'].values.tolist()]


unique_labels = set()

for lb in labels:
  [unique_labels.add(i) for i in lb if i not in unique_labels]

print(unique_labels)

{'I-ADE', 'B-Strength', 'B-Dosage', 'I-Reason', 'I-Dosage', 'B-Route', 'I-Strength', 'B-Duration', 'B-Frequency', 'I-Route', 'I-Duration', 'B-ADE', 'B-Reason', 'I-Drug', 'O', 'B-Form', 'I-Frequency', 'I-Form', 'B-Drug'}


In [14]:
# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}
print(labels_to_ids)

{'B-ADE': 0, 'B-Dosage': 1, 'B-Drug': 2, 'B-Duration': 3, 'B-Form': 4, 'B-Frequency': 5, 'B-Reason': 6, 'B-Route': 7, 'B-Strength': 8, 'I-ADE': 9, 'I-Dosage': 10, 'I-Drug': 11, 'I-Duration': 12, 'I-Form': 13, 'I-Frequency': 14, 'I-Reason': 15, 'I-Route': 16, 'I-Strength': 17, 'O': 18}


In [15]:
def formatted_df_cl(df):
  df['sentence'] = df[[
      'text_file_name',
      'original_word',
      'label']].groupby(
          ['text_file_name'])['original_word'].transform(lambda x: ' '.join(x))

  df['word_labels'] = df[[
      'text_file_name',
      'original_word',
      'label']].groupby(
          ['text_file_name'])['label'].transform(lambda x: ','.join(x))

  return df

In [16]:
df = formatted_df_cl(df)
test = formatted_df_cl(test)

In [17]:
df.iloc[0]

text_file_name                          data/training_20180910/110727.txt
sentence_line_number                                                    1
sentence_word_index                                                     0
sentence_seq                                                           NA
start_token                                                             0
end_token                                                               9
original_word                                                   Admission
word                                                            Admission
label                                                                   O
sentence                Admission Date : [ * * 2202 - 1 - 8 * * ] Disc...
word_labels             O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...
Name: 0, dtype: object

In [18]:
def sentence_level_data(df):
  sentence_level_data = df[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
  return sentence_level_data

In [20]:
sentence_level_train = sentence_level_data(df)
sentence_level_test = sentence_level_data(test)

In [23]:
data_train = sentence_level_train.copy()
data_test = sentence_level_test.copy()

In [24]:
data_train.head()

,sentence,word_labels
0,Admission Date : [ * * 2202 - 1 - 8 * * ] Disc...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,Admission Date : [ * * 2130 - 10 - 2 * * ] Dis...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,Admission Date : [ * * 2151 - 1 - 25 * * ] Dis...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,Admission Date : [ * * 2193 - 12 - 23 * * ] Di...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
4,Admission Date : [ * * 2133 - 3 - 28 * * ] Dis...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."


### It looks reasonable, let's proceed with training

In [28]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
tokenizer = AutoTokenizer.from_pretrained("yikuan8/Clinical-Longformer", add_prefix_space=True)
model = AutoModelForTokenClassification.from_pretrained(
    "yikuan8/Clinical-Longformer", num_labels=len(labels_to_ids), gradient_checkpointing=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of LongformerForTokenClassification were not initialized from the model checkpoint at yikuan8/Clinical-Longformer and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
MAX_LEN = 4096
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1
TEST_BATCH_SIZE = 2
EPOCHS = 10
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10

In [31]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.data.sentence[index].strip().split()
        word_labels = self.data.word_labels[index].split(",")

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             return_offsets_mapping=True,
                             padding='max_length',
                             is_split_into_words=True,
                             truncation=True,
                             max_length=self.max_len,
                            #  add_prefix_space=True
                            )

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels]
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item

  def __len__(self):
        return self.len

In [32]:
train_size = 0.8
train_dataset = data_train.sample(frac=train_size,random_state=200)
val_dataset = data_train.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data_train.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("Val Dataset: {}".format(val_dataset.shape))
print("Test Dataset: {}".format(data_test.shape))


training_set = dataset(train_dataset, tokenizer, MAX_LEN)
val_set = dataset(val_dataset, tokenizer, MAX_LEN)
test_set = dataset(data_test.reset_index(drop=True), tokenizer, MAX_LEN)


FULL Dataset: (303, 2)
TRAIN Dataset: (242, 2)
Val Dataset: (61, 2)
Test Dataset: (202, 2)


In [34]:
training_set[0]

{'input_ids': tensor([    0, 18032, 10566,  ...,     1,     1,     1]),
 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]),
 'offset_mapping': tensor([[0, 0],
         [0, 9],
         [0, 4],
         ...,
         [0, 0],
         [0, 0],
         [0, 0]]),
 'labels': tensor([-100,   18,   18,  ..., -100, -100, -100])}

In [35]:
test_set[0]

{'input_ids': tensor([    0, 18032, 10566,  ...,   110, 12234,     2]),
 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1]),
 'offset_mapping': tensor([[0, 0],
         [0, 9],
         [0, 4],
         ...,
         [0, 4],
         [0, 4],
         [0, 0]]),
 'labels': tensor([-100,   18,   18,  ...,   18,   18, -100])}

In [38]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)
testing_loader = DataLoader(test_set, **test_params)



In [39]:
model.to(device)

LongformerForTokenClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
             

In [40]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(3.2752, device='cuda:0', grad_fn=<NllLossBackward0>)

In [41]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 4096, 19])

In [42]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=LEARNING_RATE, )

In [43]:
import torch.nn.functional as F


def focal_loss(logits, labels, alpha=0.25, gamma=2.0, ignore_index=-100):
    """
    logits: [batch_size, seq_len, num_labels] - model predictions
    labels: [batch_size, seq_len] - ground truth labels
    """
    ce_loss = F.cross_entropy(logits.view(-1, logits.size(-1)), labels.view(-1), reduction='none', ignore_index=ignore_index)
    pred_probs = F.softmax(logits.view(-1, logits.size(-1)), dim=-1)
    pred_class = labels.view(-1)
    gamma = gamma
    filtered = labels.view(-1) != ignore_index
    ce_loss_filtered = ce_loss[filtered]
    pred_probs_filtered = pred_probs[filtered]
    pred_class_filtered = pred_class[filtered]
    pt = pred_probs_filtered.gather(1, pred_class_filtered.unsqueeze(-1)).squeeze()
    loss = ((1 - pt) ** gamma * ce_loss_filtered).mean()  # mean over the batch

    return loss



In [46]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []

    model.train()

    for idx, batch in enumerate(training_loader):
        ids = batch['input_ids'].to(device, dtype=torch.long)
        mask = batch['attention_mask'].to(device, dtype=torch.long)
        labels = batch['labels'].to(device, dtype=torch.long)

        # Create a global attention mask with 1s for first token and ADE entity locations
        global_attention_mask = torch.zeros_like(ids)
        global_attention_mask[:, 0] = 1  # Global attention on the first token ([CLS])

        # # Assuming 'labels_to_ids' is a dictionary mapping label strings to their encoded IDs
        # b_ade_id = labels_to_ids['B-Drug']
        # i_ade_id = labels_to_ids['I-Drug']

        # # Find indices where labels are B-ADE or I-ADE
        # b_ade_indices = (labels == b_ade_id).nonzero(as_tuple=True)
        # i_ade_indices = (labels == i_ade_id).nonzero(as_tuple=True)

        # # Apply global attention to B-ADE and I-ADE locations
        # # Assuming a 2D structure for `labels`, adjust dimensions as necessary
        # for idx in b_ade_indices[0]:
        #     global_attention_mask[idx] = 1
        # for idx in i_ade_indices[0]:
        #     global_attention_mask[idx] = 1

        # Perform a forward pass to get the logits
        outputs = model(input_ids=ids, attention_mask=mask, global_attention_mask=global_attention_mask)
        logits = outputs.logits
        # Calculate the loss using focal_loss function directly
        loss = focal_loss(logits, labels)  # Assuming focal_loss is correctly implemented elsewhere

        tr_loss += loss.item()
        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100 == 0:
            loss_step = tr_loss / nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # Decoding the logits to compute accuracy
        active_logits = logits.view(-1, model.num_labels)  # Adjust according to your model's output shape
        flattened_predictions = torch.argmax(active_logits, axis=1)

        # Ignoring the predictions of the padding tokens
        active_accuracy = labels.view(-1) != -100
        active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

        labels = torch.masked_select(active_labels, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels.cpu().numpy())
        tr_preds.extend(predictions.cpu().numpy())

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=MAX_GRAD_NORM)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [47]:
torch.cuda.empty_cache()

In [49]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 3.036323308944702
Training loss per 100 training steps: 0.5513081152250271
Training loss per 100 training steps: 0.3790651059973596
Training loss epoch: 0.33613162125991886
Training accuracy epoch: 0.9045512867545524
Training epoch: 2
Training loss per 100 training steps: 0.04863933473825455
Training loss per 100 training steps: 0.08356741555196483
Training loss per 100 training steps: 0.07382493885639425
Training loss epoch: 0.07024807010032622
Training accuracy epoch: 0.9722690177696489
Training epoch: 3
Training loss per 100 training steps: 0.02900957688689232
Training loss per 100 training steps: 0.04483641416645876
Training loss per 100 training steps: 0.04387062910709076
Training loss epoch: 0.04272075025061692
Training accuracy epoch: 0.9780107124097148
Training epoch: 4
Training loss per 100 training steps: 0.004501887131482363
Training loss per 100 training steps: 0.0324083151812158
Training loss per 100 training steps: 0

In [50]:
def valid(model, testing_loader):
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps = 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            ids = batch['input_ids'].to(device, dtype=torch.long)
            mask = batch['attention_mask'].to(device, dtype=torch.long)
            labels = batch['labels'].to(device, dtype=torch.long)
            global_attention_mask = torch.zeros_like(ids)
            global_attention_mask[:, 0] = 1  # Apply global attention to the first token


            outputs = model(input_ids=ids, attention_mask=mask, global_attention_mask=global_attention_mask)
            loss = focal_loss(outputs.logits, labels)

            eval_loss += loss.item()
            nb_eval_steps += 1

            if idx % 100 == 0:
                print(f"Validation loss per 100 evaluation steps: {eval_loss / nb_eval_steps}")

            active_logits = outputs.logits.view(-1, model.num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1)
            active_accuracy = labels.view(-1) != -100
            active_labels = torch.masked_select(labels.view(-1), active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(active_labels.cpu().numpy())
            eval_preds.extend(predictions.cpu().numpy())

    # Calculate and print final evaluation metrics
    eval_accuracy = accuracy_score(eval_labels, eval_preds)
    print(f"Validation Loss: {eval_loss / nb_eval_steps}")
    print(f"Validation Accuracy: {eval_accuracy}")

    # Optionally return detailed evaluation metrics
    return eval_labels, eval_preds


In [52]:
labels, predictions = valid(model, val_loader)


Validation loss per 100 evaluation steps: 0.024418480694293976
Validation Loss: 0.020247583578581172
Validation Accuracy: 0.9867474256332872


In [53]:
string_labels = [ids_to_labels[id] for id in labels]
string_predictions = [ids_to_labels[id] for id in predictions]
# since `seqeval` expects a list of lists (one per sentence), adjust accordingly
string_labels = [string_labels]
string_predictions = [string_predictions]

from seqeval.metrics import classification_report

print(classification_report(string_labels, string_predictions))

              precision    recall  f1-score   support

         ADE       0.44      0.58      0.50       130
      Dosage       0.91      0.89      0.90       550
        Drug       0.93      0.93      0.93      2412
    Duration       0.75      0.75      0.75        96
        Form       0.92      0.90      0.91       792
   Frequency       0.86      0.88      0.87       865
      Reason       0.68      0.61      0.64       545
       Route       0.91      0.91      0.91       724
    Strength       0.95      0.95      0.95       932

   micro avg       0.89      0.88      0.88      7046
   macro avg       0.82      0.82      0.82      7046
weighted avg       0.89      0.88      0.88      7046



In [54]:
test_labels, test_predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.04302993416786194
Validation loss per 100 evaluation steps: 0.02427766437522403
Validation Loss: 0.02427766437522403
Validation Accuracy: 0.98336468328611


In [57]:
string_labels = [ids_to_labels[id] for id in test_labels]
string_predictions = [ids_to_labels[id] for id in test_predictions]

# since `seqeval` expects a list of lists (one per sentence), adjust accordingly
string_labels = [string_labels]
string_predictions = [string_predictions]

from seqeval.metrics import classification_report

print(classification_report(string_labels, string_predictions))

              precision    recall  f1-score   support

         ADE       0.54      0.54      0.54       529
      Dosage       0.89      0.88      0.89      1923
        Drug       0.93      0.91      0.92      8432
    Duration       0.67      0.68      0.68       304
        Form       0.92      0.91      0.91      3014
   Frequency       0.80      0.81      0.80      2918
      Reason       0.62      0.58      0.60      1965
       Route       0.93      0.92      0.92      2568
    Strength       0.93      0.94      0.93      3114

   micro avg       0.87      0.86      0.87     24767
   macro avg       0.80      0.80      0.80     24767
weighted avg       0.87      0.86      0.87     24767



### Save

In [63]:
import os

directory = "/content/drive/MyDrive/266_final/clinical_longformer_final"

if not os.path.exists(directory):
    os.makedirs(directory)

tokenizer.save_vocabulary(directory)
model.save_pretrained(directory, safe_serialization=True)